In [54]:
import pandas as pd

In [55]:
df = pd.read_csv("links.csv", index_col=[0])
df = df.drop_duplicates( keep='last')
df = df.reset_index(drop=True)

In [56]:
df['true_link'] = 'https://ingatlan.com/' + df['link']
df['true_link'] = df['true_link'].astype('str')

In [65]:
df['location'] = df['link'].str.split('/').str[1]
df['type'] = df['link'].str.split('/').str[3]

In [67]:
df

,link,true_link,location,type
0,/xiii-ker/elado+lakas/tegla-epitesu-lakas/3293...,https://ingatlan.com//xiii-ker/elado+lakas/teg...,xiii-ker,tegla-epitesu-lakas
1,/viii-ker/elado+lakas/tegla-epitesu-lakas/3298...,https://ingatlan.com//viii-ker/elado+lakas/teg...,viii-ker,tegla-epitesu-lakas
2,/xiv-ker/elado+lakas/tegla-epitesu-lakas/32879321,https://ingatlan.com//xiv-ker/elado+lakas/tegl...,xiv-ker,tegla-epitesu-lakas
3,/xiii-ker/elado+lakas/tegla-epitesu-lakas/3300...,https://ingatlan.com//xiii-ker/elado+lakas/teg...,xiii-ker,tegla-epitesu-lakas
4,/vii-ker/elado+lakas/tegla-epitesu-lakas/32772342,https://ingatlan.com//vii-ker/elado+lakas/tegl...,vii-ker,tegla-epitesu-lakas
...,...,...,...,...
51145,/gyor-revfalu/elado+lakas/tegla-epitesu-lakas/...,https://ingatlan.com//gyor-revfalu/elado+lakas...,gyor-revfalu,tegla-epitesu-lakas
51146,/vii-ker/elado+lakas/tegla-epitesu-lakas/5196070,https://ingatlan.com//vii-ker/elado+lakas/tegl...,vii-ker,tegla-epitesu-lakas
51147,/vi-ker/elado+lakas/tegla-epitesu-lakas/5069451,https://ingatlan.com//vi-ker/elado+lakas/tegla...,vi-ker,tegla-epitesu-lakas
51148,/viii-ker/elado+lakas/tegla-epitesu-lakas/4563923,https://ingatlan.com//viii-ker/elado+lakas/teg...,viii-ker,tegla-epitesu-lakas
